# Logistic Regression Notebook
[Return to project overview](final_project_overview.ipynb)

### Andrew Larimer, Deepak Nagaraj, Daniel Olmstead, Michael Winton (W207-4-Summer 2018 Final Project)

In [ ]:
# import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from util import our_train_test_split

# set default options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

%matplotlib inline

In [ ]:
from functools import partial
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.2
RANDOM_STATE = 207
PASSNYC_LABELS = [0,1]
our_train_test_split = partial(train_test_split,
                               stratify=y,
                               test_size=TEST_SIZE,
                               random_state=RANDOM_STATE)
merged_df = pd.read_csv('data_merged/combined_data_2018-07-18.csv')
X = merged_df.loc[:, ~merged_df.columns.isin(['high_registrations'])]
y = merged_df.loc[:, merged_df.columns.isin(['high_registrations'])]

train_data, test_data, train_labels, test_labels = our_train_test_split(X, y)
train_data

In [ ]:
from sklearn.linear_model import LogisticRegression
c_values = {'C': [0.010, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 1.000, 2.000, 3.000, 4.000]}
for c in c_values['C']:
    log = LogisticRegression(C=c, penalty='l2', random_state=207)
    log.fit(train_data, train_labels)
    log_predicted_labels = log.predict(test_data)
    log_score = metrics.f1_score(test_labels, log_predicted_labels, average='weighted')
    print("F1 score for C={}: {:.4f}".format(c, log_score))
